# Tree Methods

In this lecture we will code along with some data and test out 3 different tree methods:

* A single decision tree
* A random forest
* A gradient boosted tree classifier
    
We will be using a college dataset to try to classify colleges as Private or Public based off these features:

    Private A factor with levels No and Yes indicating private or public university
    Apps Number of applications received
    Accept Number of applications accepted
    Enroll Number of new students enrolled
    Top10perc Pct. new students from top 10% of H.S. class
    Top25perc Pct. new students from top 25% of H.S. class
    F.Undergrad Number of fulltime undergraduates
    P.Undergrad Number of parttime undergraduates
    Outstate Out-of-state tuition
    Room.Board Room and board costs
    Books Estimated book costs
    Personal Estimated personal spending
    PhD Pct. of faculty with Ph.D.’s
    Terminal Pct. of faculty with terminal degree
    S.F.Ratio Student/faculty ratio
    perc.alumni Pct. alumni who donate
    Expend Instructional expenditure per student
    Grad.Rate Graduation rate

In [20]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [21]:
# Load training data
data = spark.read.csv('../data/College.csv',inferSchema=True,header=True)

In [22]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [25]:
data.show(vertical=True)

-RECORD 0---------------------------
 School      | Abilene Christian... 
 Private     | Yes                  
 Apps        | 1660                 
 Accept      | 1232                 
 Enroll      | 721                  
 Top10perc   | 23                   
 Top25perc   | 52                   
 F_Undergrad | 2885                 
 P_Undergrad | 537                  
 Outstate    | 7440                 
 Room_Board  | 3300                 
 Books       | 450                  
 Personal    | 2200                 
 PhD         | 70                   
 Terminal    | 78                   
 S_F_Ratio   | 18.1                 
 perc_alumni | 12                   
 Expend      | 7041                 
 Grad_Rate   | 60                   
-RECORD 1---------------------------
 School      | Adelphi University   
 Private     | Yes                  
 Apps        | 2186                 
 Accept      | 1924                 
 Enroll      | 512                  
 Top10perc   | 16                   
 

### Spark Formatting of Data

In [26]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [28]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [30]:
output = assembler.transform(data)

Deal with Private column being "yes" or "no"

In [9]:
from pyspark.ml.feature import StringIndexer

In [31]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [32]:
final_data = output_fixed.select("features",'PrivateIndex')

In [33]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

### The Classifiers

In [37]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

Create all three models:

In [38]:
# Use mostly defaults to make this comparison "fair"

dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

Train all three models:

In [39]:
# Train the models (its three models, so it might take some time)
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Model Comparison

Let's compare each of these models!

In [40]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [42]:
dtc_predictions.show()

+--------------------+------------+-------------+-----------+----------+
|            features|PrivateIndex|rawPrediction|probability|prediction|
+--------------------+------------+-------------+-----------+----------+
|[81.0,72.0,51.0,3...|         0.0|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|[100.0,90.0,35.0,...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[152.0,128.0,75.0...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[167.0,130.0,46.0...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[174.0,146.0,88.0...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[191.0,165.0,63.0...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[245.0,208.0,125....|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[247.0,189.0,100....|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[268.0,253.0,103....|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[283.0,201.0,97.0...|         0.0|  [284.0,0.0]|  [1.0,0.0]|       0.0|
|[325.0,260.0,86.0...|         0.0|  [284.0,0.0]|  

**Evaluation Metrics:**

In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [44]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [45]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [46]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 91.49%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 93.62%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 91.49%
